In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import psycopg2
from tqdm import tqdm
from psycopg2 import sql

In [ ]:
login_url="https://proxy.bmstu.ru:8443/cas/login?service=https%3A%2F%2Fproxy.bmstu.ru%3A8443%2Fcas%2Foauth2.0%2FcallbackAuthorize%3Fclient_name%3DCasOAuthClient%26client_id%3DEU"
target_url="https://eu.bmstu.ru/modules/progress3/"
username="" # Ввести логин от KAS
password="" # Ввести пароль от KAS
output_file='groups.csv'

db_config = {
    'host': 'localhost',
    'database': 'KALM_Db2',
    'user': 'postgres',
    'password': '6257',
    'port': '5432'
}

In [3]:
# 1. Настройка Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Режим без отображения браузера
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

In [5]:
driver = webdriver.Chrome(options=options)

try:
  conn = psycopg2.connect(**db_config)
  cursor = conn.cursor()
  
  # 2. Авторизация
  driver.get(login_url)
  
  # Ждем появления полей ввода
  WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.NAME, "username"))
  ).send_keys(username)
  
  driver.find_element(By.NAME, "password").send_keys(password)
  
  # Нажимаем кнопку входа (адаптируйте селектор под ваш сайт)
  driver.find_element(By.NAME, 'submit').click()
  time.sleep(3)  # Ждем загрузки
  
  # 3. Проверка успешной авторизации
  if "login" in driver.current_url.lower():
    raise Exception("Ошибка авторизации! Проверьте логин/пароль.")
  
  # 4. Переход на целевую страницу
  driver.get(target_url)
  WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "eu-tree-leaf")))
  
  # 5. Парсинг данных
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  groups = []
  
  for leaf in tqdm(soup.find_all('li', class_='eu-tree-leaf'), desc='Получение групп'):
    link = leaf.find('a')
    if link:
      groups.append({
        'Группа': link.get_text(strip=True),
        'Ссылка': link.get('href')
      })

  cursor.execute("""SELECT DISTINCT "Group_Name" FROM public."Groups";""")
  
  db_groups = {row[0] for row in cursor.fetchall()}
        
  # Фильтрация групп
  filtered_groups = [
      group for group in groups 
      if group['Группа'] in db_groups
  ]
  
  with open('students.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Группа', 'ФИО', 'Номер_зачетки'])
    writer.writeheader()

    for group in tqdm(filtered_groups, desc='Получение студентов'):
      driver.get('https://eu.bmstu.ru' + group['Ссылка'])
      WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "tbody")))
      
      table = driver.find_element(By.CLASS_NAME, 'progress_students').find_element(By.TAG_NAME, 'tbody')
      table_html = table.get_attribute("innerHTML")
      
      soup = BeautifulSoup(table_html, 'html.parser')
      students = []
      
      for row in soup.find_all('tr'):
        if ('tr-disabled' in row.get('class', [])):
          continue
        if row:
          writer.writerow({
            'Группа': group['Группа'],
            'ФИО': ' '.join(row.find('span', class_ = 'fio_3').get_text(strip=True).split()),
            'Номер_зачетки': row.find('td', class_ = 'hc3').get_text(strip=True)
          })
  
  # 6. Сохранение в CSV
  with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Группа', 'Ссылка'])
    writer.writeheader()
    writer.writerows(filtered_groups)
  
  print(f"Успешно сохранено {len(filtered_groups)} групп в {output_file}")
  
except Exception as e:
  print(f"Ошибка: {str(e)}")

finally:
  driver.quit()

Получение студентов: 100%|██████████| 1563/1563 [17:47<00:00,  1.46it/s]


Успешно сохранено 1563 групп в groups.csv
